In [1]:
import numpy as np
from random import randint

In [2]:
'''Rosembrock Function'''
def Fun(x):
    fun=0
    a=1.0
    b=100.0
    fun = (a-x[0])**2 + b*(x[1]-x[0]**2)**2
    return fun

In [3]:
''' 
FOBJ gera vetorialmente os valores com a funcao objetivo escalar
em outras palavras gera para populacao o valor da funcao 
a ser otimizada
'''
def FOBJ(X):
    rows = X.shape[0]
    fobj=np.zeros(rows)
    for i in range(rows):
        fobj[i]=Fun(X[i,])
    return fobj

In [4]:
'''Enxame retorna a populacao aleatoria com todos valores entre MIN e MAX'''
def Enxame(PAR,NPAR,MAX,MIN):
    x=np.zeros((NPAR, len(MAX)))
    for j in range(len(MAX)):
        for i in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

In [5]:
''' 
Fitness gera vetorialmente os valores com a funcao fitness
equivale a quantidade de nectar da fonte
'''
def FIT(X):
    rows = X.shape[0]
    fit=np.zeros(rows)
    fit=np.copy(FOBJ(X))
    for i in range(rows):
        if(fit[i]>=0):
            fit[i]=1/(fit[i]+1)
        else:
            fit[i]=1+abs(fit[i])
    return fit

In [6]:
'''
probablilidade usada para roleta q atualizacao das onlookers 
semelhante ao genetico
'''

def PROB(x):
    rows=x.shape[0]
    fit=np.zeros(rows)
    prob=np.zeros(rows)
    fit=FIT(x)
    soma=np.sum(fit)
    
    for i in range(rows):
        prob[i] = fit[i]/soma

    return prob

In [7]:
'''
Obtem a melhor escolha greedy search
'''
def GetBest(x,xbest):
    ycal=FOBJ(x)
    best=np.argmin(ycal)
    yref=Fun(xbest)
    if(yref<ycal[best]):
        GBEST=xbest
    else:
        GBEST=x[best]
    return GBEST
    

In [8]:
'''
employed bee phase
'''
def EmployedBee(x,PAR,NPAR) :
    #x= Enxame(PAR,NPAR,MAX,MIN)
    V=np.zeros((NPAR, len(MAX)))
    xold=np.zeros((NPAR, len(MAX)))
    for j in range(PAR):
        for i in range(NPAR):
            r=np.random.randint(NPAR, size=PAR)
            k=r[j]
            if(k==i):
                if(k<(NPAR-1)):
                    k=k+1
                else:
                    k=k-1
            V[i,j]=np.copy(x[k,j])
    YCAL=FOBJ(x)
    YV=FOBJ(V)
    for i in range(NPAR):
        if(YV[i]<YCAL[i]):
            x[i,]=np.copy(V[i,])

    return x

In [9]:
'''
onlooker bee phase
'''
def OnlookerBee(trial,xo,x):
    rows = x.shape[0]
    cols = x.shape[1]
    prob=np.zeros(rows)
    V=np.copy(xo)
    prob=PROB(x)
    ycal=FOBJ(x)
    best=np.argmin(ycal)
    
    for i in range(rows):
        
        rand=np.random.random() # este aleatorio vai ser utilizado no metodo da roleta
        prob[i]=0.9*prob[i]+0.1 # esta melhora no prob nao esta no original
        
        if(rand<prob[i]): # metodo da roleta so faz a mudanca se o sorteio for favoravel
            for j in range(cols):
                r=np.random.randint(NPAR, size=PAR)
                k=r[j]
                if(k==i):
                    if(k<(NPAR-1)):
                        k=k+1
                    else:
                        k=k-1
                fi=np.random.uniform(low=-1.0, high=1.0, size=None)
                rd=randint(0, (rows-1))
                V[i,j]=xo[i,j]+(x[rd,j] -x[k,j])*fi # precisa mudar esta eq: variantes metodo
                if(Fun(V[i,])<Fun(x[i,])):
                    xo[i,]=np.copy(V[i,])
                    x[i,]=np.copy(V[i,])
                    trial[i]=0
                else:
                    trial[i]=trial[i]+1
    #return trial,xo,x        
    return trial,xo,x

In [10]:
'''
scout bee phase
'''
def ScoutBee(trial,x,ntrail,MAX,MIN):
    rows = x.shape[0]
    cols = x.shape[1]
    for i in range(rows):
        if(trial[i]>=ntrail):
            trial[i]=0
            for j in range(cols):
                x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x,trial

In [11]:

'''
Metodo que alterna e coordena o uso employers,onlooker, scout bees
e greedy search

'''
def ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN):
    x=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee
    xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa onlooker bee
    trial=np.zeros(NPAR) # inicializa contador p abandonar fonte de alimento
    xbest=np.copy(x[0,]) #inicializa xbest
    xbest=GetBest(x,xbest)
    xbest=GetBest(xo,xbest)
    for i in range(ITE):
        x=EmployedBee(x,PAR,NPAR)
        xbest=GetBest(x,xbest)
        trial,xo,x=OnlookerBee(trial,xo,x)
        xbest=GetBest(xo,xbest)
        x,trial=ScoutBee(trial,x,ntrail,MAX,MIN)
        xbest=GetBest(x,xbest)
    return xbest  

In [12]:
NPAR=50 #PARTICULAS
ITE=300 #ITERACOES
PAR=2 #NUM DE PARAMETROS A SER OTIMIZADOS
MAX=[2,2] # MAXIMO DE CADA PARAMETRO
MIN=[-2,-2] # MINIMO DE CADA PARAMETRO
ntrail=50 #numero de buscas ate abandonar uma fonte de alimento

xbest=ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN)
print(xbest)
print(Fun(xbest))

[0.66435352 0.43989364]
0.11287522603575754
